## Llamar API

In [80]:
import requests
import pandas as pd

lat = 4.57937
lon = -74.21682
api_key = 'SBUdfKRVGJEmElRKmmHGGHgC4nTSzgH2'

url = 'https://api.windy.com/api/point-forecast/v2'
headers = {'Content-Type': 'application/json'}

payload = {
    "lat": lat,
    "lon": lon,
    "model": "gfs",
    "parameters": [
        'rh', 'pressure', 'windGust', 'precip', 
        'cape', 'lclouds', 'convPrecip' 
    ],
    "levels": ["surface"],
    "key": api_key
}

response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    data = response.json()
    df = pd.DataFrame({k: v for k, v in data.items() if k != 'units'})
    print(df.head())
else:
    print("Error:", response.status_code)
    print(response.text)

              ts  rh-surface  pressure-surface  gust-surface  \
0  1762030800000   85.967313     101595.992700      2.515978   
1  1762041600000   85.142830     101253.756415      2.035879   
2  1762052400000   86.719535     101782.316505      2.843617   
3  1762063200000   87.485448     101578.533798      2.484131   
4  1762074000000   91.928000     101733.660089      2.478069   

   past3hprecip-surface  cape-surface  lclouds-surface  \
0              0.002059      0.000000        31.147870   
1              0.003753     45.976556        17.475907   
2              0.000000      0.000000         8.844894   
3              0.003301    455.176793         0.489595   
4              0.000000     76.957666         6.763744   

   past3hconvprecip-surface                                            warning  
0                  0.001782  The testing API version is for development pur...  
1                  0.000165  The testing API version is for development pur...  
2                  0.00

## Generar Datos

In [205]:
import pandas as pd

# Cargar el dato base real
base = pd.read_csv('dato_base_normal.csv')

# Asegurar que tenga las columnas esperadas
columnas_modelo = [
    'past3hprecip-surface',
    'past3hconvprecip-surface',
    'rh-surface',
    'pressure-surface',
    'gust-surface',
    'cape-surface',
    'lclouds-surface'
]

base = base[columnas_modelo]

import numpy as np

# Función para perturbar el dato base y simular inundación
def simular_inundacion(base, n=500):
    simulados = []
    for _ in range(n):
        punto = base.iloc[0].copy()
        punto['past3hprecip-surface'] = np.random.uniform(0.2, 0.5)  # lluvia intensa
        punto['past3hconvprecip-surface'] = np.random.uniform(0.2, 0.5)
        punto['rh-surface'] = np.random.uniform(95, 100)  # saturación
        punto['pressure-surface'] = np.random.uniform(95000, 97000)  # presión baja
        punto['gust-surface'] = np.random.uniform(8, 15)  # ráfagas fuertes
        punto['cape-surface'] = np.random.uniform(400, 1000)  # alta inestabilidad
        punto['lclouds-surface'] = np.random.uniform(80, 100)  # cielo cubierto
        punto['inundacion'] = 1
        simulados.append(punto)
    return pd.DataFrame(simulados)

def simular_normales(base, n=500):
    simulados = []
    for _ in range(n):
        punto = base.iloc[0].copy()
        punto['past3hprecip-surface'] = np.random.uniform(0.0, 0.001)
        punto['past3hconvprecip-surface'] = np.random.uniform(0.0, 0.001)
        punto['rh-surface'] = np.random.uniform(30, 60)
        punto['pressure-surface'] = np.random.uniform(99500, 102000)
        punto['gust-surface'] = np.random.uniform(0, 1.5)
        punto['cape-surface'] = np.random.uniform(0, 20)
        punto['lclouds-surface'] = np.random.uniform(0, 20)
        punto['inundacion'] = 0
        simulados.append(punto)
    return pd.DataFrame(simulados)

In [206]:
import pandas as pd

# Cargar ambos archivos
normales = pd.read_csv('datos_simulados_normales.csv')
inundacion = pd.read_csv('datos_simulados_inundacion.csv')

# Unirlos en un solo DataFrame
df_completo = pd.concat([normales, inundacion], ignore_index=True)

# Mezclar aleatoriamente las filas
df_completo = df_completo.sample(frac=1, random_state=42).reset_index(drop=True)

# Guardar dataset final
df_completo.to_csv('dataset_entrenamiento_inundacion.csv', index=False)
print("✅ Dataset final guardado como dataset_entrenamiento_inundacion.csv")

✅ Dataset final guardado como dataset_entrenamiento_inundacion.csv


## Modelo

In [207]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import joblib

# Generar dataset completo
df = pd.concat([
    simular_normales(base, n=500),
    simular_inundacion(base, n=500)
], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)

X = df[columnas_modelo]
y = df['inundacion']

# Entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipe_svc = make_pipeline(StandardScaler(), SVC(probability=True))
pipe_svc.fit(X_train, y_train)

# Evaluación
print(f"✅ Accuracy en test: {pipe_svc.score(X_test, y_test):.3f}")

# Guardar modelo
joblib.dump(pipe_svc, 'modelo_inundacion.pkl')
print("✅ Modelo reentrenado y guardado como modelo_inundacion.pkl")

✅ Accuracy en test: 1.000
✅ Modelo reentrenado y guardado como modelo_inundacion.pkl


## Predicción

In [208]:
import requests
import pandas as pd
import joblib

# === 1. Parámetros de ubicación y API ===
# La María
lat = 4.597956
lon = -74.201885
api_key = 'SBUdfKRVGJEmElRKmmHGGHgC4nTSzgH2'

url = 'https://api.windy.com/api/point-forecast/v2'
headers = {'Content-Type': 'application/json'}
parameters = ['rh', 'pressure', 'windGust', 'precip', 'cape', 'lclouds', 'convPrecip']
levels = ['surface']

payload = {
    "lat": lat,
    "lon": lon,
    "model": "gfs",
    "parameters": parameters,
    "levels": levels,
    "key": api_key
}

# === 2. Consultar Windy API ===
response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    data = response.json()

    # === 3. Crear DataFrame con nombres originales del API ===
    df_raw = pd.DataFrame({k: v for k, v in data.items() if k != 'units'})

    # === 4. Renombrar columnas para que coincidan con el modelo entrenado ===
    renombrar = {
        'precip': 'past3hprecip-surface',
        'convPrecip': 'past3hconvprecip-surface',
        'rh': 'rh-surface',
        'pressure': 'pressure-surface',
        'windGust': 'gust-surface',
        'cape': 'cape-surface',
        'lclouds': 'lclouds-surface'
    }

    df_renombrado = df_raw.rename(columns=renombrar)

    # === 5. Seleccionar la primera fila y columnas esperadas por el modelo ===
    columnas_modelo = list(renombrar.values())
    df_single = df_renombrado.iloc[[0]][columnas_modelo]

    # === 6. Cargar el modelo entrenado ===
    modelo = joblib.load('modelo_inundacion.pkl')

    # === 7. Predecir probabilidad de inundación ===
    probabilidad = modelo.predict_proba(df_single)[0][1]
    print(f"Probabilidad de inundación en La Maria ({lat}, {lon}): {probabilidad:.2f}")

else:
    print("❌ Error al consultar Windy API:", response.status_code)
    print(response.text)

Probabilidad de inundación en La Maria (4.597956, -74.201885): 0.27


In [209]:
import requests
import pandas as pd
import joblib

# === 1. Parámetros de ubicación y API ===
# Danubio
lat = 4.590101
lon = -74.224665
api_key = 'SBUdfKRVGJEmElRKmmHGGHgC4nTSzgH2'

url = 'https://api.windy.com/api/point-forecast/v2'
headers = {'Content-Type': 'application/json'}
parameters = ['rh', 'pressure', 'windGust', 'precip', 'cape', 'lclouds', 'convPrecip']
levels = ['surface']

payload = {
    "lat": lat,
    "lon": lon,
    "model": "gfs",
    "parameters": parameters,
    "levels": levels,
    "key": api_key
}

# === 2. Consultar Windy API ===
response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    data = response.json()

    # === 3. Crear DataFrame con nombres originales del API ===
    df_raw = pd.DataFrame({k: v for k, v in data.items() if k != 'units'})

    # === 4. Renombrar columnas para que coincidan con el modelo entrenado ===
    renombrar = {
        'precip': 'past3hprecip-surface',
        'convPrecip': 'past3hconvprecip-surface',
        'rh': 'rh-surface',
        'pressure': 'pressure-surface',
        'windGust': 'gust-surface',
        'cape': 'cape-surface',
        'lclouds': 'lclouds-surface'
    }

    df_renombrado = df_raw.rename(columns=renombrar)

    # === 5. Seleccionar la primera fila y columnas esperadas por el modelo ===
    columnas_modelo = list(renombrar.values())
    df_single = df_renombrado.iloc[[0]][columnas_modelo]

    # === 6. Cargar el modelo entrenado ===
    modelo = joblib.load('modelo_inundacion.pkl')

    # === 7. Predecir probabilidad de inundación ===
    probabilidad = modelo.predict_proba(df_single)[0][1]
    print(f"Probabilidad de inundación en Danubio ({lat}, {lon}): {probabilidad:.2f}")

else:
    print("❌ Error al consultar Windy API:", response.status_code)
    print(response.text)

Probabilidad de inundación en Danubio (4.590101, -74.224665): 0.01


In [242]:
import pandas as pd
import numpy as np

# Simulación de entorno de riesgo moderado
simulacion = pd.DataFrame([{
    'past3hprecip-surface': np.random.uniform(0.05, 0.15),         # lluvia significativa
    'past3hconvprecip-surface': np.random.uniform(0.05, 0.15),     # convectiva activa
    'rh-surface': np.random.uniform(85, 95),                       # humedad alta
    'pressure-surface': np.random.uniform(97000, 98500),           # presión algo baja
    'gust-surface': np.random.uniform(4, 8),                       # ráfagas moderadas
    'cape-surface': np.random.uniform(150, 400),                   # inestabilidad moderada
    'lclouds-surface': np.random.uniform(50, 80)                   # nubosidad baja elevada
}])

print("✅ Muestra simulada de entorno de riesgo moderado:")
print(simulacion)

✅ Muestra simulada de entorno de riesgo moderado:
   past3hprecip-surface  past3hconvprecip-surface  rh-surface  \
0              0.063238                  0.124337   91.048778   

   pressure-surface  gust-surface  cape-surface  lclouds-surface  
0      97312.773381      7.386431    155.528663        70.425789  


In [243]:
import joblib

# Cargar modelo entrenado
modelo = joblib.load('modelo_inundacion.pkl')

# Predecir probabilidad
probabilidad = modelo.predict_proba(simulacion)[0][1]
print(f"Probabilidad simulada de inundación: {probabilidad:.2f}")

Probabilidad simulada de inundación: 0.80


## API

In [245]:
!pip install fastapi uvicorn joblib requests pandas scikit-learn

Defaulting to user installation because normal site-packages is not writeable
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ------------------------------ --------- 1.6/2.0 MB 8.1 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 7.7 MB/s eta 0:00:00
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)

   ------------- -------------------------- 3/9 [uvicorn]
   ------------- -------------------------- 3/9 [uvicorn]
   ------------- -------------------------- 3/9 [uvicorn]
   ------------- -------------------------- 3/9 [uvicorn]
   ------------- -------------------------- 3/9 [uvicorn]
   ------------- -------------------------- 3/9 [uvicorn]
   ----------------- ---------------------- 4/9 [typing-inspection]
   ---------------------- ----------------- 5/9 [starlette]
   ---------------------- ----------------- 5/9 [starlette]
   ---------------------- -------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [246]:
from fastapi import FastAPI
from pydantic import BaseModel
import requests
import pandas as pd
import joblib

app = FastAPI()

# === Cargar modelo entrenado ===
modelo = joblib.load('modelo_inundacion.pkl')

# === Configuración Windy API ===
API_KEY = 'SBUdfKRVGJEmElRKmmHGGHgC4nTSzgH2'
WINDY_URL = 'https://api.windy.com/api/point-forecast/v2'
HEADERS = {'Content-Type': 'application/json'}
PARAMS = ['rh', 'pressure', 'windGust', 'precip', 'cape', 'lclouds', 'convPrecip']
LEVELS = ['surface']
RENOMBRAR = {
    'precip': 'past3hprecip-surface',
    'convPrecip': 'past3hconvprecip-surface',
    'rh': 'rh-surface',
    'pressure': 'pressure-surface',
    'windGust': 'gust-surface',
    'cape': 'cape-surface',
    'lclouds': 'lclouds-surface'
}
COLUMNAS_MODELO = list(RENOMBRAR.values())

# === Entrada esperada ===
class Ubicacion(BaseModel):
    lat: float
    lon: float

# === Endpoint principal ===
@app.post("/inundacion")
def predecir_inundacion(ubicacion: Ubicacion):
    payload = {
        "lat": ubicacion.lat,
        "lon": ubicacion.lon,
        "model": "gfs",
        "parameters": PARAMS,
        "levels": LEVELS,
        "key": API_KEY
    }

    response = requests.post(WINDY_URL, json=payload, headers=HEADERS)
    if response.status_code != 200:
        return {"error": f"API Windy falló: {response.status_code}", "detalle": response.text}

    data = response.json()
    df_raw = pd.DataFrame({k: v for k, v in data.items() if k != 'units'})
    df_renombrado = df_raw.rename(columns=RENOMBRAR)
    df_single = df_renombrado.iloc[[0]][COLUMNAS_MODELO]

    probabilidad = modelo.predict_proba(df_single)[0][1]
    return {
        "lat": ubicacion.lat,
        "lon": ubicacion.lon,
        "probabilidad_inundacion": round(probabilidad, 4)
    }

In [281]:
import requests

base_url = "http://127.0.0.1:8000"

endpoints = {
    "La María": "/la_maria",
    "Danubio": "/danubio",
    "Simulación arriesgada": "/simulacion_arriesgada",
    "Simulación peligrosa": "/simulacion_peligrosa",
    "Datos Windy La Maria": "/parametros_la_maria",
    "Datos Windy Danubio": "/parametros_danubio"
}

for nombre, endpoint in endpoints.items():
    url = base_url + endpoint
    print(f"🔍 Consultando {nombre} → {url}")
    try:
        response = requests.get(url)
        response.raise_for_status()
        resultado = response.json()
        print(f"✅ Resultado: {resultado}\n")
    except Exception as e:
        print(f"❌ Error en {nombre}: {e}\n")

🔍 Consultando La María → http://127.0.0.1:8000/la_maria
✅ Resultado: [0.024, 'baja']

🔍 Consultando Danubio → http://127.0.0.1:8000/danubio
✅ Resultado: [0.0654, 'baja']

🔍 Consultando Simulación arriesgada → http://127.0.0.1:8000/simulacion_arriesgada
✅ Resultado: [0.7848, 'alta']

🔍 Consultando Simulación peligrosa → http://127.0.0.1:8000/simulacion_peligrosa
✅ Resultado: [1.0, 'alta']

🔍 Consultando Datos Windy La Maria → http://127.0.0.1:8000/parametros_la_maria
✅ Resultado: {'ts': 1762030800000, 'rh-surface': 93.55531662953575, 'pressure-surface': 101604.19633411664, 'gust-surface': 1.6273103457180633, 'past3hprecip-surface': 1.864375743888589e-05, 'cape-surface': 0, 'lclouds-surface': 11.906979583117108, 'past3hconvprecip-surface': 0.00017980956450086778, 'warning': 'T'}

🔍 Consultando Datos Windy Danubio → http://127.0.0.1:8000/parametros_danubio
✅ Resultado: {'ts': 1762030800000, 'rh-surface': 79.52814993133342, 'pressure-surface': 101898.89967750176, 'gust-surface': 1.66737212